# dataset link
https://www.kaggle.com/datasets/andrewmvd/animal-faces


In [1]:
import torch
from torch import nn
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import os


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
"""
loop through images and labels then create dataframe
"""

image_path = []
labels = []
data_folder = 'afhq'

for i in os.listdir(data_folder):
    for label in os.listdir(f"{data_folder}/{i}"):
        for image in os.listdir(f"{data_folder}/{i}/{label}"):
            image_path.append(f"{data_folder}/{i}/{label}/{image}")
            labels.append(label)

df = pd.DataFrame(zip(image_path, labels), columns=["path", "label"])

In [4]:
df.head()

,path,label
0,afhq/val/cat/pixabay_cat_000440.jpg,cat
1,afhq/val/cat/pixabay_cat_000099.jpg,cat
2,afhq/val/cat/pixabay_cat_000531.jpg,cat
3,afhq/val/cat/pixabay_cat_002156.jpg,cat
4,afhq/val/cat/flickr_cat_000802.jpg,cat


In [5]:
df["label"].unique()

array(['cat', 'dog', 'wild'], dtype=object)

In [6]:
"""
split data for training, testing and validation
"""

train = df.sample(frac=0.7)
test = df.drop(train.index)
val = test.sample(frac=0.5)
test = test.drop(val.index)

print(train.shape)
print(test.shape)
print(val.shape)

(11291, 2)
(2419, 2)
(2420, 2)


In [7]:
"""
encode our labels
"""
label_encoder = LabelEncoder()
label_encoder.fit(df["label"])

LabelEncoder()

In [8]:
"""
create a transformer to process images
"""

transform = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

In [79]:
class CustomImageDataset(Dataset):
    """
    create a dataset class to convert data to torch dataset
    """
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = torch.tensor(
            label_encoder.transform(dataframe['label'])
        ).to(device)

    def __len__(self):
        """
        return length of dataframe
        """
        return self.dataframe.shape[0]

    def __getitem__(self, idx):
        """
        get and open image using index
        """
        label = self.labels[idx]
        img_path = self.dataframe.iloc[idx, 0]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img).to(device)
        return img, label
        

In [82]:
train_data = CustomImageDataset(dataframe=train, transform=transform)
test_data = CustomImageDataset(dataframe=test, transform=transform)
val_data = CustomImageDataset(dataframe=val, transform=transform)

In [83]:
img, label = train_data.__getitem__(2)

In [85]:
label_encoder.inverse_transform([torch.Tensor.cpu(label)])

array(['cat'], dtype=object)

In [89]:
LR = 1e-4
BATCH_SIZE = 50
EPOCHS = 10


In [ ]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)
